In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import io
import cv2
import random
import os
import shutil

import matplotlib.pyplot as plt

In [ ]:
DATA_PATH = "/kaggle/input/cattle-weight-detection-model-dataset-12k/www.acmeai.tech Dataset - BMGF-LivestockWeight-CV"
JSON_PATH_REAR = os.path.join(DATA_PATH, "Vector/B4/Rear/data/coco_b4_rear.json")
IMAGE_DIR_REAR = os.path.join(DATA_PATH, "Vector/B4/Rear/data/images")

JSON_PATH_SIDE = os.path.join(DATA_PATH, "Vector/B4/Side/data/coco_b4_side.json")
IMAGE_DIR_SIDE = os.path.join(DATA_PATH, "Vector/B4/Side/data/images")

# Function to load and display the first image along with its keypoints
def visualize_first_image(json_path, image_dir):
    with open(json_path, 'r') as file:
        coco_data = json.load(file)

    first_image_filename = coco_data['images'][0]['file_name']
    first_image_keypoints = coco_data['annotations'][0]['keypoints']
    keypoint_x_vals = first_image_keypoints[0::3]
    keypoint_y_vals = first_image_keypoints[1::3]

    image_path = os.path.join(image_dir, first_image_filename)
    image = Image.open(image_path)

    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.scatter(keypoint_x_vals, keypoint_y_vals, c='r', marker='x')
    plt.title(f"First Image with Keypoints from {json_path.split('/')[-2]}")
    plt.show()
# Visualize for both Rear and Side
visualize_first_image(JSON_PATH_REAR, IMAGE_DIR_REAR)
visualize_first_image(JSON_PATH_SIDE, IMAGE_DIR_SIDE)

In [ ]:
def visualize_first_image(json_path, image_dir, final_ls,size):
    with open(json_path, 'r') as file:
        coco_data = json.load(file)
    for j in range(size):
        first_image_filename = coco_data['images'][j]['file_name']
        first_image_keypoints = coco_data['annotations'][j]['keypoints']
        keypoint_x_vals = first_image_keypoints[0::3]
        keypoint_y_vals = first_image_keypoints[1::3]

        image_path = os.path.join(image_dir, first_image_filename)
    #     image = Image.open(image_path)

    #     plt.figure(figsize=(10,10))
    #     plt.imshow(image)
    #     plt.scatter(keypoint_x_vals, keypoint_y_vals, c='r', marker='x')
    #     plt.title(f"First Image with Keypoints from {json_path.split('/')[-2]}")
    #     plt.show()
        ls = []
        for i in range(len(keypoint_x_vals)):
            ls.append([keypoint_x_vals[i],keypoint_y_vals[i]])
        ls.append(image_path)
        final_ls.append(ls)

In [ ]:
final_ls = []
JSON_PATH_REAR = os.path.join(DATA_PATH, "Vector/B4/Rear/data/coco_b4_rear.json")
IMAGE_DIR_REAR = os.path.join(DATA_PATH, "Vector/B4/Rear/data/images")
visualize_first_image(JSON_PATH_REAR, IMAGE_DIR_REAR,final_ls,size=1900)
JSON_PATH_REAR = os.path.join(DATA_PATH, "Vector/B3/Rear/data/COCO_B3_rear.json")
IMAGE_DIR_REAR = os.path.join(DATA_PATH, "Vector/B3/Rear/data/images")
visualize_first_image(JSON_PATH_REAR, IMAGE_DIR_REAR,final_ls,size=2500)

In [ ]:
import pandas as pd
columns = ['one','two','three','four','image_path']
df = pd.DataFrame(final_ls,columns = columns)

In [ ]:
df.info()

In [ ]:
train_dataset = df

In [ ]:
import random
from skimage import io
import cv2
plt.figure(figsize = (10,10))
for plot_index in range(9):
    index  = random.randint(0, len(train_dataset) - 1)
    image  = io.imread(train_dataset['image_path'][index],1)
    points = train_dataset.loc[index][:-1]
    points = [np.uint(points[x]) for x in range(len(points))]
    for point_index in range(len(points)):
        image = cv2.circle(image, points[point_index] , radius=1, color=1, thickness=-1)
    plt.subplot(3,3, plot_index + 1)
    plt.imshow(image)
    plt.axis('off')

In [ ]:
os.mkdir('data')
os.mkdir('data/images')
os.mkdir('data/labels')
os.mkdir('data/images/train')
os.mkdir('data/images/val')
os.mkdir('data/labels/train')
os.mkdir('data/labels/val')

In [ ]:
# split into train and val
for index in range(len(train_dataset) - 200):
    file_name = os.path.split(train_dataset.loc[index]['image_path'])[1].split('.')[0]
    inst_data = [0, 0.5,0.5,0.5,0.5]
    label_path = os.path.join('/kaggle/working/data/labels/train/', (file_name + '.txt'))
    image_path = os.path.join('/kaggle/working/data/images/train/', (file_name + '.jpg'))
    binary_image = cv2.imread(train_dataset.loc[index]['image_path'], cv2.IMREAD_GRAYSCALE)
    H, W = binary_image.shape
    slc = train_dataset.iloc[index].to_numpy(copy=True)[:-1]
    for i in range(4):
        inst_data.extend([slc[i][0]/W, slc[i][1]/H])
    np.array(inst_data,dtype = object).tofile(f'{label_path}', sep=' ')
    shutil.copyfile(train_dataset.loc[index]['image_path'], image_path)

In [ ]:
for index in range((len(train_dataset) - 200), len(train_dataset)):
    file_name = os.path.split(train_dataset.loc[index]['image_path'])[1].split('.')[0]
    inst_data = [0, 0.5,0.5,0.5,0.5]
    label_path = os.path.join('/kaggle/working/data/labels/val/', (file_name + '.txt'))
    image_path = os.path.join('/kaggle/working/data/images/val/', (file_name + '.jpg'))
    binary_image = cv2.imread(train_dataset.loc[index]['image_path'], cv2.IMREAD_GRAYSCALE)
    H, W = binary_image.shape
    slc = train_dataset.iloc[index].to_numpy(copy=True)[:-1]
    for i in range(4):
        inst_data.extend([slc[i][0]/W, slc[i][1]/H])
    np.array(inst_data,dtype = object).tofile(f'{label_path}', sep=' ')
    shutil.copyfile(train_dataset.loc[index]['image_path'], image_path)

In [ ]:
!pip install ultralytics

In [ ]:
import wandb
api_key = 'ccdf912083f622610063ad0ff31defb3972f6ee7'
wandb.login(key=api_key,relogin=True)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8l-pose.pt')  # load a pretrained model (recommended for training)
results = model.train(data='/kaggle/input/dummy-yaml/data_keypoint_model.yaml',
                      epochs=50, imgsz=640, cache = True, pretrained = True)